In [ ]:
# install dependencies
!pip install langchain
!pip install langchain_ai
!pip install langchain_openai
!pip install langchain_community
!pip install langchain_pinecone
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

ERROR: Ignored the following versions that require a different python version: 0.1.0 Requires-Python <4.0,>=3.11; 0.1.1 Requires-Python <4.0,>=3.11
ERROR: Could not find a version that satisfies the requirement langchain_ai (from versions: none)
ERROR: No matching distribution found for langchain_ai
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.2 MB/s eta 0:0

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
# connect to google drive
import os
from google.colab import drive,userdata

drive.mount('/content/gdrive')
os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY']=userdata.get('PINECONE_API_KEY')

Mounted at /content/gdrive


In [ ]:
# load file and add to index
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_community.document_loaders import TextLoader,DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter

index_name='unc-partner-prototype-limited'
embeddings=OpenAIEmbeddings(model='text-embedding-3-small')

loader=DirectoryLoader('/content/gdrive/MyDrive/partner-prototype/docs/txt/use', glob='**/*.txt', loader_cls=TextLoader)
doc=loader.load()

text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=0)
doc=text_splitter.split_documents(doc)

docs_vectorstore=PineconeVectorStore.from_documents(doc,index_name=index_name,embedding=embeddings)

In [ ]:
# run the chat
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings,ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

index_name='unc-partner-prototype'
embeddings=OpenAIEmbeddings(model='text-embedding-3-small')
vs=PineconeVectorStore(index_name=index_name,embedding=embeddings)

pt=ChatPromptTemplate.from_messages([
    ('system','{context}'),
    ("user", "{prompt}")
])
model=ChatOpenAI()
parser=StrOutputParser()
chain=pt | model | parser


while True:
    print('\u001b[1mHow can I help? \u001b[0m')
    prompt=input('\t\t')
    if prompt=="exit" or prompt=="":
      print("Goodbye.")
      break


    result=vs.similarity_search(prompt,k=2)
    context="\n".join([source.page_content for source in result])

    response=chain.invoke({
        'context':context,
        "prompt":prompt
    })
    print(response)


How can I help? 
I recommend reaching out to local organizations and agencies that specialize in providing housing resources for teens with severe mental health conditions. Here are some resources you can explore:

1. **NAMI (National Alliance on Mental Illness)**: Contact your local NAMI affiliate to inquire about housing resources for teens with severe mental health conditions. NAMI provides support, education, and advocacy for individuals and families affected by mental illness.

2. **Local Mental Health Agencies**: Contact your local mental health agency or department of health and human services. They may have information on housing resources specifically tailored for teens with severe mental health conditions.

3. **Centers for Independent Living (CILs)**: Check the ILRU Directory of Centers for Independent Living to find local peer-based housing and support organizations that may offer resources for teens with mental health conditions.

4. **Community Behavioral Health Services*